In [88]:
#Import dependencies
import pandas as pd

#Import files from resources folder in my new repository 
school_data_csv = "Resources/schools_complete.csv"
student_data_csv = "Resources/students_complete.csv"

#Read school and student csvs and store into dataframe
school_data = pd.read_csv(school_data_csv)
student_data = pd.read_csv(student_data_csv)

#Merge the datasets using a left join to make one single dataset 
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name","school_name"])

In [89]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [90]:
#Set variable for total number of schools
unique_schools = school_data_complete["school_name"].nunique()

#Set variable for total number of students
students_count = school_data_complete["student_name"].count()

In [91]:
#Set variable for calculating average math score
average_math_score = school_data_complete["math_score"].mean()

#Set variable for calculating average reading score
average_reading_score = school_data_complete["reading_score"].mean()

In [146]:
#Get individual budget for each unique school
school_data_complete.loc[:, ["school_name", "budget"]].head()

inv_budget = school_data_complete['budget'].unique()

#Get total budget by summing individual budgets
total_budget = inv_budget.sum()

In [147]:
#Get count of students passing math, passing reading, and passing both
passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70, ["student_name"]]

passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70, ["student_name"]]

passing_both = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) &
                                        (school_data_complete["math_score"] >= 70), ["student_name"]]


In [148]:
#Set count of variables as seperate variables to find the percentage of students passing
pass_reading = passing_reading.count()
pass_math = passing_math.count()
pass_both = passing_both.count()

In [149]:
#Calculate percentage of students passing reading, math, and both
pass_reading_percent = ((pass_reading)/(students_count)*100)

pass_math_percent = ((pass_math)/(students_count)*100)

pass_both_percent = ((pass_both)/(students_count)*100)

In [150]:
#Create new data frame for District Summary
district_df = pd.DataFrame({"Total schools":[unique_schools],
                           "Total students": [students_count],
                            "Total budget":[total_budget],
                           "Average math score":[average_math_score],
                           "Average reading score":[average_reading_score],
                           "% Passing Math": [pass_math_percent],
                           "% Passing Reading": [pass_reading_percent],
                           "% Overall Passing":[pass_both_percent]})

#Format percentages, total students, and total budget for summary table
district_df["% Passing Math"]=district_df["% Passing Math"].astype(float)
district_df["% Passing Math"]=district_df["% Passing Math"].map("{:,.2f}%".format)

district_df["% Passing Reading"]=district_df["% Passing Reading"].astype(float)
district_df["% Passing Reading"]=district_df["% Passing Reading"].map("{:,.2f}%".format)

district_df["% Overall Passing"]=district_df["% Overall Passing"].astype(float)
district_df["% Overall Passing"]=district_df["% Overall Passing"].map("{:,.2f}%".format)

district_df["Total bugdet"]=district_df["Total budget"].astype(float)
district_df["Total budget"]=district_df["Total budget"].map("${:,.2f}".format)

district_df["Total students"]=district_df["Total students"].map("{:,.0f}".format)

#Delete extra budget column
del district_df['Total bugdet']

district_df

,Total schools,Total students,Total budget,Average math score,Average reading score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


In [153]:
#Copy original dataframe to work with for school summary
school_data_df = school_data_complete.copy()

#Drop extra column for school ID
del school_data_df['School ID']

In [154]:
#Calculate and add new column for per student budget
school_data_df['Per student budger']= school_data_df['budget']/school_data_df['size']

In [155]:
#Calculate average reading and math score per school using groupyby
average_reading_math_schools = school_data_complete.groupby("school_name")['reading_score','math_score'].mean().reset_index()

<ipython-input-155-946fc4bfe6f7>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_reading_math_schools = school_data_complete.groupby("school_name")['reading_score','math_score'].mean().reset_index()


In [156]:
#Merge in average reading and math scores to school data DF
school_data_df = school_data_df.merge(average_reading_math_schools, on='school_name', how='outer')

In [157]:
#Rename columns to Average Reading and Average Math
school_data_df.rename(columns={'reading_score_y':'Average Reading Score','math_score_y':'Average Math Score'})

,Student ID,student_name,gender,grade,school_name,reading_score_x,math_score_x,type,size,budget,Per student budger,Average Reading Score,Average Math Score
0,0,Paul Bradley,M,9th,Huang High School,66,79,District,2917,1910635,655.0,81.182722,76.629414
1,1,Victor Smith,M,12th,Huang High School,94,61,District,2917,1910635,655.0,81.182722,76.629414
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,District,2917,1910635,655.0,81.182722,76.629414
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,District,2917,1910635,655.0,81.182722,76.629414
4,4,Bonnie Ray,F,9th,Huang High School,97,84,District,2917,1910635,655.0,81.182722,76.629414
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,Charter,1635,1043130,638.0,83.848930,83.418349
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,Charter,1635,1043130,638.0,83.848930,83.418349
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,Charter,1635,1043130,638.0,83.848930,83.418349
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,Charter,1635,1043130,638.0,83.848930,83.418349


In [158]:
#Create variables to calculate amount of students passing reading, math, and passing both
sum_pass_r = school_data_complete[school_data_complete['reading_score'] >= 70]

sum_pass_m = school_data_complete[school_data_complete['math_score'] >= 70]

sum_pass_both = school_data_complete[(school_data_complete["reading_score"] >= 70) &
                                        (school_data_complete["math_score"] >= 70)]

In [161]:
#Use variables created to calcuate the amount of students passing for each school
count_pass_r = sum_pass_r.groupby('school_name')['reading_score'].count().reset_index()

count_pass_m = sum_pass_m.groupby('school_name')['math_score'].count().reset_index()

count_pass_both = sum_pass_both.groupby('school_name')['student_name'].count().reset_index()

In [163]:
#Merge in count of students passing math, reading, and both grouped by school
passing_count = count_pass_m.merge(count_pass_r, on="school_name", how="inner")

passing_count_all = passing_count.merge(count_pass_both, on="school_name",how="inner")
passing_count_all.rename(columns = {'math_score': 'Math Pass Count', 'reading_score': 'Reading Pass Count',
                                    'student_name':'Pass both count'})

,school_name,Math Pass Count,Reading Pass Count,Pass both count
0,Bailey High School,3318,4077,2719
1,Cabrera High School,1749,1803,1697
2,Figueroa High School,1946,2381,1569
3,Ford High School,1871,2172,1487
4,Griffin High School,1371,1426,1330
5,Hernandez High School,3094,3748,2481
6,Holden High School,395,411,381
7,Huang High School,1916,2372,1561
8,Johnson High School,3145,3867,2549
9,Pena High School,910,923,871


In [164]:
#Merge in passing counts to school data DF
school_data_df = school_data_df.merge(passing_count_all, on='school_name', how='outer')

In [165]:
#Calculate percent passing and add new column with those values in school data DF

school_data_df['% Passing Math'] = (school_data_df['math_score']/school_data_df['size'])*100

school_data_df['% Passing Reading'] = (school_data_df['reading_score']/school_data_df['size'])*100

school_data_df['% Overall Passing'] = (school_data_df['student_name_y']/school_data_df['size'])*100

school_data_df.head()

,Student ID,student_name_x,gender,grade,school_name,reading_score_x,math_score_x,type,size,budget,Per student budger,reading_score_y,math_score_y,math_score,reading_score,student_name_y,% Passing Math,% Passing Reading,% Overall Passing
0,0,Paul Bradley,M,9th,Huang High School,66,79,District,2917,1910635,655.0,81.182722,76.629414,1916,2372,1561,65.683922,81.316421,53.513884
1,1,Victor Smith,M,12th,Huang High School,94,61,District,2917,1910635,655.0,81.182722,76.629414,1916,2372,1561,65.683922,81.316421,53.513884
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,District,2917,1910635,655.0,81.182722,76.629414,1916,2372,1561,65.683922,81.316421,53.513884
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,District,2917,1910635,655.0,81.182722,76.629414,1916,2372,1561,65.683922,81.316421,53.513884
4,4,Bonnie Ray,F,9th,Huang High School,97,84,District,2917,1910635,655.0,81.182722,76.629414,1916,2372,1561,65.683922,81.316421,53.513884


In [166]:
#Delete count columns that we used to calculate percentages because we no longer need them
del school_data_df['math_score']
del school_data_df['reading_score']
del school_data_df['student_name_y']

In [177]:
summary_df = school_data_df[['school_name',
       'type', 'size', 'budget',
       'Per student budger', 'reading_score_y', 'math_score_y',
       '% Passing Math', '% Passing Reading','% Overall Passing']]

summary_df.head()

,school_name,type,size,budget,Per student budger,reading_score_y,math_score_y,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
1,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
2,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
3,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
4,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884


In [183]:
school_summary_df = summary_df.drop_duplicates(subset='school_name', keep='first', inplace=False,ignore_index=False)


In [184]:
school_summary_df = school_summary_df.rename(columns={"school_name":"School Name","type":"School Type","size":"Total Students",
                           "budget":"Total School Budget","Per student budger": "Per Student Budget",
                          "reading_score_y":"Average Reading Score","math_score_y":"Average Math Score"})

In [187]:
school_summary_df.set_index(['School Name'])

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,89.892107
Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.680064,81.933280,54.642283
Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.505855,96.252927,89.227166


In [188]:
#Sort top 5 performing schools by overall passing
top_df = school_summary_df.sort_values("% Overall Passing", ascending=False)
top_df.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
16013,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
37535,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,93.272171,97.308869,90.948012
12262,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
13730,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567
23274,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541


In [189]:
#Sort botttom 5 performing schools by overall passing 
low_df = school_summary_df.sort_values("% Overall Passing", ascending=True)
low_df.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
26036,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,66.366592,80.220055,52.988247
2917,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
7627,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
30035,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,66.057551,81.222432,53.539172


In [191]:
#Copy original data frame to work with for generating math and reading scores for each school/grade
math_df = school_data_complete.copy()
math_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [195]:
#Set variables for holding scores for each grade
ninth_grade = math_df.loc[math_df["grade"] == "9th"]

tenth_grade = math_df.loc[math_df["grade"] == "10th"]

eleven_grade = math_df.loc[math_df["grade"] == "11th"]

twelve_grade = math_df.loc[math_df["grade"] == "12th"]

In [196]:
#Use above variables to generate a series of average math scores per grade for each school
ninth_math_score = ninth_grade.groupby(["school_name"]).mean()["math_score"]

tenth_math_score = tenth_grade.groupby(["school_name"]).mean()["math_score"]

eleven_math_score = eleven_grade.groupby(["school_name"]).mean()["math_score"]

twelve_math_score = twelve_grade.groupby(["school_name"]).mean()["math_score"]

In [197]:
#Create new data frame with average math grades per grade grouped by school
average_math_grades_df = pd.DataFrame({"9th": ninth_math_score,
                                      "10th": tenth_math_score,
                                      "11th": eleven_math_score,
                                      "12th": twelve_math_score})
average_math_grades_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [198]:
#Using same grade variables used for getting average math scores, create variables for
#finding average reading score per grade grouped by schools
ninth_read_score = ninth_grade.groupby(["school_name"]).mean()["reading_score"]

tenth_read_score = tenth_grade.groupby(["school_name"]).mean()["reading_score"]

eleven_read_score = eleven_grade.groupby(["school_name"]).mean()["reading_score"]

twelve_read_score = twelve_grade.groupby(["school_name"]).mean()["reading_score"]


In [199]:
average_reading_grades_df = pd.DataFrame({"9th": ninth_read_score,
                                      "10th": tenth_read_score,
                                      "11th": eleven_read_score,
                                      "12th": twelve_read_score})
average_reading_grades_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [129]:
spending_df = new_df.copy()
spending_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
2917,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
5866,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,89.892107
7627,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
12262,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
13730,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567
16013,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
17871,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.680064,81.933280,54.642283
22847,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.505855,96.252927,89.227166
23274,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541


In [130]:
bins = [0,585,630,645,676]
group_names = ["<$584","$585-629","$630-644","$645-675"]


In [131]:
spending_df["Spending Ranges Per Student"] = pd.cut(spending_df["Per Student Budget"], bins, labels=group_names,
                                                   include_lowest=True)

spending_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges Per Student
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884,$645-675
2917,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476,$630-644
5866,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,89.892107,$585-629
7627,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508,$645-675
12262,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455,$585-629
13730,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567,<$584
16013,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769,<$584
17871,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.680064,81.933280,54.642283,$585-629
22847,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.505855,96.252927,89.227166,<$584
23274,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541,$585-629


In [132]:
av_read_bin = spending_df.groupby(["Spending Ranges Per Student"]).mean()["Average Reading Score"]
av_read_bin

Spending Ranges Per Student
<$584       83.933814
$585-629    83.155286
$630-644    81.624473
$645-675    81.027843
Name: Average Reading Score, dtype: float64

In [133]:
av_math_bin = spending_df.groupby(["Spending Ranges Per Student"]).mean()["Average Math Score"]
av_math_bin

Spending Ranges Per Student
<$584       83.455399
$585-629    81.899826
$630-644    78.518855
$645-675    76.997210
Name: Average Math Score, dtype: float64

In [134]:
av_pass_m_bin = spending_df.groupby(["Spending Ranges Per Student"]).mean()["% Passing Math"]
av_pass_m_bin

av_pass_r_bin = spending_df.groupby(["Spending Ranges Per Student"]).mean()["% Passing Reading"]
av_pass_r_bin

av_pass_o_bin = spending_df.groupby(["Spending Ranges Per Student"]).mean()["% Overall Passing"]
av_pass_o_bin

Spending Ranges Per Student
<$584       90.369459
$585-629    81.418596
$630-644    62.857656
$645-675    53.526855
Name: % Overall Passing, dtype: float64

In [135]:
scores_by_spending_df = pd.DataFrame({"Average Math Score": av_math_bin,
                                     "Average Reading Score": av_read_bin,
                                     "% Passing Math": av_pass_m_bin,
                                     "% Passing Reading": av_pass_r_bin,
                                     "% Overall Passing": av_pass_o_bin})

scores_by_spending_df["Average Math Score"]=scores_by_spending_df["Average Math Score"].map("{:,.2f}".format)

scores_by_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges Per Student,,,,,
<$584,83.46,83.933814,93.460096,96.610877,90.369459
$585-629,81.90,83.155286,87.133538,92.718205,81.418596
$630-644,78.52,81.624473,73.484209,84.391793,62.857656
$645-675,77.00,81.027843,66.164813,81.133951,53.526855


In [136]:
size_df = new_df.copy()
size_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
2917,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
5866,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,89.892107
7627,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
12262,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
13730,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567
16013,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
17871,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.680064,81.933280,54.642283
22847,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.505855,96.252927,89.227166
23274,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541


In [137]:
bins = [0, 1001, 2001, 5001]
group_names = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]

In [138]:
size_df["School Size"] = pd.cut(size_df["Total Students"], bins, labels=group_names,
                                                   include_lowest=True)

size_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884,Large(2000-5000)
2917,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476,Large(2000-5000)
5866,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,89.892107,Medium(1000-2000)
7627,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508,Large(2000-5000)
12262,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455,Medium(1000-2000)
13730,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567,Large(2000-5000)
16013,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769,Medium(1000-2000)
17871,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.680064,81.933280,54.642283,Large(2000-5000)
22847,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.505855,96.252927,89.227166,Small(<1000)
23274,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541,Small(<1000)


In [139]:
av_read_size = size_df.groupby(["School Size"]).mean()["Average Reading Score"]
av_read_size

av_math_size = size_df.groupby(["School Size"]).mean()["Average Math Score"]
av_math_size

School Size
Small(<1000)         83.821598
Medium(1000-2000)    83.374684
Large(2000-5000)     77.746417
Name: Average Math Score, dtype: float64

In [140]:
av_pass_m_size = size_df.groupby(["School Size"]).mean()["% Passing Math"]
av_pass_m_size

av_pass_r_size = size_df.groupby(["School Size"]).mean()["% Passing Reading"]
av_pass_r_size

av_pass_o_size = size_df.groupby(["School Size"]).mean()["% Overall Passing"]
av_pass_o_size

School Size
Small(<1000)         89.883853
Medium(1000-2000)    90.621535
Large(2000-5000)     58.286003
Name: % Overall Passing, dtype: float64

In [141]:
school_size_df = pd.DataFrame({"Average Math Score": av_math_size,
                              "Average Reading Score": av_read_size,
                              "% Passing Math": av_pass_m_size,
                              "% Passing Reading": av_pass_r_size,
                              "% Overall Passing": av_pass_o_size})
school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [142]:
type_df = new_df.copy()
type_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.683922,81.316421,53.513884
2917,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
5866,Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.867121,95.854628,89.892107
7627,Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.752967,80.862999,53.527508
12262,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.392371,97.138965,90.599455
13730,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.867718,96.539641,90.582567
16013,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
17871,Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.680064,81.933280,54.642283
22847,Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.505855,96.252927,89.227166
23274,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,94.594595,95.945946,90.540541


In [143]:
#charter_school = type_df.loc[type_df["School Type"] == "Charter"]

#district_school = type_df.loc[type_df["School Type"] == "District"]


In [144]:
c_read_score = type_df.groupby(["School Type"]).mean()["Average Reading Score"]

c_math_score = type_df.groupby(["School Type"]).mean()["Average Math Score"]

c_pass_math = type_df.groupby(["School Type"]).mean()["% Passing Math"]

c_pass_read = type_df.groupby(["School Type"]).mean()["% Passing Reading"]

c_pass_o = type_df.groupby(["School Type"]).mean()["% Overall Passing"]


In [145]:
scores_by_type_df = pd.DataFrame({"Average Math Score": c_math_score,
                                 "Average Reading Score": c_read_score,
                                 "% Passing Math": c_pass_math,
                                 "% Passing Reading": c_pass_read,
                                 "% Overall Passing": c_pass_o})
scores_by_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
